# Everything is in the last cell !

current features:
- engagement rate
- average likes per post
- average comments per post
- not yet: virality rate, but would be cool (a draft of the formula is in the cell)


also: i created a dictionary of company_name -> 2-d dataframe with time-series (sorted) data for that company
called company_tables, index e.g. by company_tables['Nike']

In [1]:
import pandas as pd
data_path = '../data/skylab_instagram_datathon_dataset.csv'

data = pd.read_csv(data_path, delimiter=';')  # Specify the delimiter (e.g., comma)

data["period_end_date"] = pd.to_datetime(data["period_end_date"])

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [61]:
print(data.columns)

Index(['period', 'period_end_date', 'compset_group', 'compset',
       'business_entity_doing_business_as_name', 'legal_entity_name',
       'domicile_country_name', 'ultimate_parent_legal_entity_name',
       'primary_exchange_name', 'calculation_type', 'followers', 'pictures',
       'videos', 'comments', 'likes'],
      dtype='object')


In [5]:
important_cols = ["followers","pictures","videos","comments","likes"]
funnylist = []

for group_idx in data["compset_group"].unique():
    filtered_df = data[data["compset_group"] == group_idx]
    filtered_sum = filtered_df.isna().sum()
    funnylist.append((group_idx,
                      sum(filtered_sum[important_cols])/len(filtered_df)))

funnydict = {idx:cumulative for idx,cumulative in funnylist}

In [ ]:
import matplotlib.pyplot as plt

def calculate_yearly_means(df, col, category_column, category_value, period: str="yearly"):
    filtered_df = df[df[category_column] == category_value]
    
    # Convert the index to datetime if it's not already
    if period == "yearly":
        filtered_df["time"] = pd.to_datetime(filtered_df["period_end_date"]).dt.year
    elif period == "montly":
        filtered_df["time"] = pd.to_datetime(filtered_df["period_end_date"]).dt.month

    yearly_means = filtered_df[col].groupby(filtered_df["time"]).mean()
    
    return yearly_means


metrics = ["followers", "pictures", "likes", "videos", "comments"]

for col in metrics:
    compset_groups = data["compset_group"].unique()
    num_columns = len(compset_groups)
    fig, axes = plt.subplots(num_columns, 1, figsize=(10, 6*num_columns))
    plt.title(col)
    for i, cat in enumerate(compset_groups):
        cat_means = calculate_yearly_means(df=data, col="followers", category_column="compset", category_value=cat)
        cat_means.plot(ax=axes[i], title=cat)
        axes[i].set_ylabel('Mean' + ' ' + col.capitalize())
        axes[i].set_xlabel('Year')

plt.tight_layout()
plt.show()

In [58]:
# Sort the data by company name and period end date
impute_test = data.sort_values(by=['business_entity_doing_business_as_name', 'period_end_date'])

# Group the data by company name
grouped = impute_test.groupby('business_entity_doing_business_as_name')

# For each company, process the data to handle duplicates and compset variations
company_tables = {}
for name, group in grouped:
    # Drop the company name column as it's redundant in individual company dataframes
    group = group.drop(columns='business_entity_doing_business_as_name')
    group.drop_duplicates()
    
    # Group by period_end_date and aggregate
    group = group.groupby('period_end_date').agg({
        'compset': lambda x: list(set(x)),  # Convert compset values to a set to remove duplicates, then to a list
        'followers': 'mean',  # Assuming followers should be averaged
        'pictures': 'sum',  # Summing up the pictures
        'videos': 'sum',    # Summing up the videos
        'comments': 'sum',  # Summing up the comments
        'likes': 'sum'      # Summing up the likes
    }).reset_index()

    # Store the processed DataFrame back to the company_tables dictionary
    company_tables[name] = group


In [50]:
# Now each company's DataFrame is updated in the company_tables dictionary
# For example, to view the data for 'Nike':
nike_data = company_tables['Nike']
print(nike_data)

    period_end_date                                            compset  \
0        2015-01-03  [US Softlines Analyst Interest List, Sportswea...   
1        2015-01-10  [US Softlines Analyst Interest List, Sportswea...   
2        2015-01-17  [US Softlines Analyst Interest List, Sportswea...   
3        2015-01-24  [US Softlines Analyst Interest List, Sportswea...   
4        2015-01-31  [US Softlines Analyst Interest List, Sportswea...   
..              ...                                                ...   
450      2023-08-19  [US Softlines Analyst Interest List, Sportswea...   
451      2023-08-26  [US Softlines Analyst Interest List, Sportswea...   
452      2023-09-02  [US Softlines Analyst Interest List, Sportswea...   
453      2023-09-09  [US Softlines Analyst Interest List, Sportswea...   
454      2023-09-16  [US Softlines Analyst Interest List, Sportswea...   

       followers  pictures  videos  comments       likes  
0     24471305.0     438.0     0.0  274617.0  348986

In [2]:
import pandas as pd
data_path = '../data/skylab_instagram_datathon_dataset.csv'

data = pd.read_csv(data_path, delimiter=';')  # Specify the delimiter (e.g., comma)

data["period_end_date"] = pd.to_datetime(data["period_end_date"])

# Sort the data by company name and period end date
impute_test = data.sort_values(by=['business_entity_doing_business_as_name', 'period_end_date'])

# Group the data by company name
grouped = impute_test.groupby('business_entity_doing_business_as_name')

# For each company, process the data to handle duplicates and compset variations
company_tables = {}
for name, group in grouped:
    # Drop the company name column as it's redundant in individual company dataframes
    group = group.drop(columns='business_entity_doing_business_as_name')
    group.drop_duplicates()
    
    # Group by period_end_date and aggregate
    group = group.groupby('period_end_date').agg({
        'compset': lambda x: list(set(x)),  # Convert compset values to a set to remove duplicates, then to a list
        'followers': 'mean',  # Assuming followers should be averaged
        'pictures': 'sum',  # Summing up the pictures
        'videos': 'sum',    # Summing up the videos
        'comments': 'sum',  # Summing up the comments
        'likes': 'sum'      # Summing up the likes
    }).reset_index()

    # Store the processed DataFrame back to the company_tables dictionary
    company_tables[name] = group


# Iterate over each company in the company_tables
for company, df in company_tables.items():
    # Ensure the data is sorted by date - assuming 'period_end_date' is the weekly identifier
    df = df.sort_values(by='period_end_date')

    # Group by 'period_end_date' to handle data week by week
    weekly_data = df.groupby('period_end_date').agg({
        'likes': 'sum',
        'comments': 'sum',
        'followers': 'mean',  # Assuming 'followers' is stable across the week, we take the mean
        'pictures': 'sum',    # Summing up the number of pictures
        'videos': 'sum'       # Summing up the number of videos
    }).reset_index()

    # Calculate the average engagement per post for the baseline
    weekly_data['average_engagement_per_post'] = (weekly_data['likes'] + weekly_data['comments']) / (weekly_data['pictures'] + weekly_data['videos'] + 0.001)

    # Calculate the engagement rate for each week
    weekly_data['engagement_rate'] = (weekly_data['likes'] + weekly_data['comments']) / weekly_data['followers'] * 100

    # Calculate the like-to-comment ratio for each week
    # I actually think this feature does not make sense
    #weekly_data['like_to_comment_ratio'] = weekly_data['likes'] / (weekly_data['comments'] + 0.001)

    # Calculate the average likes and comments per post for each week
    weekly_data['average_likes_per_post'] = weekly_data['likes'] / (weekly_data['pictures'] + weekly_data['videos'] + 0.001)
    weekly_data['average_comments_per_post'] = weekly_data['comments'] / (weekly_data['pictures'] + weekly_data['videos'] + 0.001)

    # Calculate the virality rate for each week
    # Virality rate = (engagement per post this week / average engagement per post baseline) * 100
    # weekly_data['virality_rate'] = (weekly_data['average_engagement_per_post'] / weekly_data['average_engagement_per_post'].mean()) * 100

    # Merge the calculated weekly metrics back to the original dataframe
    df = pd.merge(df, weekly_data[['period_end_date', 'engagement_rate', 'average_likes_per_post', 'average_comments_per_post']], on='period_end_date', how='left')

    # Update the company_tables dictionary with the modified DataFrame
    company_tables[company] = df

# Optionally, to check the results for a specific company, e.g., 'Nike'
nike_data = company_tables['Nike']
print(nike_data[['period_end_date', 'pictures', 'likes', 'comments', 'followers', 'engagement_rate', 'average_likes_per_post', 'average_comments_per_post']].head())


  period_end_date  pictures       likes  comments   followers  \
0      2015-01-03     438.0  34898691.0  274617.0  24471305.0   
1      2015-01-10     465.0  39738612.0  321669.0  25299746.0   
2      2015-01-17     471.0  43148382.0  393432.0  26143731.0   
3      2015-01-24     447.0  40263060.0  383376.0  26851899.0   
4      2015-01-31     399.0  36760893.0  411510.0  27483940.0   

   engagement_rate  average_likes_per_post  average_comments_per_post  
0       143.732866            79677.194801                 626.978021  
1       158.342621            84911.382668                 687.325454  
2       166.547820            89892.275224                 819.648292  
3       151.372668            88295.990579                 840.734998  
4       135.251361            90100.007108                1008.600469  


In [2]:
company_tables

{'24S':     period_end_date                                            compset  \
 0        2017-05-06  [Ecommerce Apparel Retail, Ecommerce Luxury Ap...   
 1        2017-05-13  [Ecommerce Luxury Apparel Retail, US Softlines...   
 2        2017-05-20  [Ecommerce Apparel Retail, Ecommerce Luxury Ap...   
 3        2017-05-27  [Ecommerce Apparel Retail, Ecommerce Luxury Ap...   
 4        2017-06-03  [Ecommerce Apparel Retail, Ecommerce Luxury Ap...   
 ..              ...                                                ...   
 328      2023-08-19  [Ecommerce Luxury Apparel Retail, US Softlines...   
 329      2023-08-26  [Ecommerce Luxury Apparel Retail, US Softlines...   
 330      2023-09-02  [Ecommerce Apparel Retail, Ecommerce Luxury Ap...   
 331      2023-09-09  [Ecommerce Luxury Apparel Retail, US Softlines...   
 332      2023-09-16  [Ecommerce Apparel Retail, Ecommerce Luxury Ap...   
 
      followers  pictures  videos  comments    likes  engagement_rate  \
 0          NaN   